In [ ]:
# %% [code]
import importlib, dmrg_runner, validate_jewel
importlib.reload(dmrg_runner)
importlib.reload(validate_jewel)
from validate_jewel import run_and_evaluate_delta

import numpy as np
import matplotlib.pyplot as plt
import os
import json

print("--- DMRG Simulation for PRL ---")
print("NOTE: This script runs multiple, larger simulations and will take significant time (potentially hours or days).")

# 1) Parameters for a “PRL-Grade” run
L_values     = [64, 96] 
bond_dims    = [64, 128, 256, 512] 
cutoffs      = [1e-10] * len(bond_dims) 
n_sweeps     = 12 
tol          = 1e-8 
fit_r_min    = 3 
fit_r_max_fac = 0.5 
outdir       = "prl_jewel_runs"

os.makedirs(outdir, exist_ok=True)

results_delta1 = {}
results_delta2 = {}
summaries_delta1 = {}
summaries_delta2 = {}

# 2) Loop over system sizes and run simulations
for L in L_values:
    print(f"\n--- Starting Simulations for L = {L} ---")
    r_max = L // 2
    r_max_fit = int(r_max * fit_r_max_fac) 
    print(f"L={L}, Max bond_dim={max(bond_dims)}, Sweeps={n_sweeps}")
    print(f"MI will be calculated up to r={r_max}. Fits will use r in [{fit_r_min}, ~{r_max_fit}].")

    # --- Run Δ=1.0 (Critical) ---
    print(f"\nRunning Δ=1.0 for L={L}...")
    props1, sum1 = run_and_evaluate_delta(
        1.0,
        L=L,
        bond_dims_schedule=bond_dims,
        cutoffs_schedule=cutoffs,
        n_sweeps=n_sweeps,
        r_max_mi=r_max,
        r_max_corr=r_max,
        fit_r_min=fit_r_min,
        fit_r_max_factor=fit_r_max_fac,
        output_dir=outdir
        # run_label=f"L{L}_delta1" # REMOVED THIS LINE
    )
    results_delta1[L] = props1
    summaries_delta1[L] = sum1

    # --- Run Δ=2.0 (Gapped) ---
    print(f"\nRunning Δ=2.0 for L={L}...")
    props2, sum2 = run_and_evaluate_delta(
        2.0,
        L=L,
        bond_dims_schedule=bond_dims,
        cutoffs_schedule=cutoffs,
        n_sweeps=n_sweeps,
        r_max_mi=r_max,
        r_max_corr=r_max,
        fit_r_min=fit_r_min,
        fit_r_max_factor=fit_r_max_fac,
        output_dir=outdir
        # run_label=f"L{L}_delta2" # REMOVED THIS LINE
    )
    results_delta2[L] = props2
    summaries_delta2[L] = sum2

    # --- Convergence Check ---
    print(f"\n--- Convergence Check for L = {L} ---")
    converged1 = summaries_delta1.get(L, {}).get('converged', 'Unknown')
    variance1 = summaries_delta1.get(L, {}).get('var_val', np.nan)
    print(f"Δ=1.0: Converged = {converged1}, Variance = {variance1:.2e}")
    if not (converged1 is True):
         print(f"⚠️ WARNING: Δ=1.0 L={L} DID NOT CONVERGE or status unknown!")

    converged2 = summaries_delta2.get(L, {}).get('converged', 'Unknown')
    variance2 = summaries_delta2.get(L, {}).get('var_val', np.nan)
    print(f"Δ=2.0: Converged = {converged2}, Variance = {variance2:.2e}")
    if not (converged2 is True):
         print(f"⚠️ WARNING: Δ=2.0 L={L} DID NOT CONVERGE or status unknown!")
    print("-" * 30)

print("\n--- All Simulations Finished ---")

# --- Save Summaries ---
summary_file = os.path.join(outdir, "all_summaries.json")
all_summaries = {'delta1': summaries_delta1, 'delta2': summaries_delta2}
try:
    with open(summary_file, 'w') as f:
        json.dump(all_summaries, f, indent=4)
    print(f"Saved run summaries to {summary_file}")
except TypeError as e:
    print(f"Could not save full summary due to non-serializable data: {e}.")

# --- Analysis & Finite-Size Scaling (Example) ---
print("\n--- Fit Exponents (X) vs 1/L for Δ=1.0 ---")
X_values = []
L_inv_values = []
for L in L_values:
    X = summaries_delta1.get(L, {}).get('fit_X_mi', np.nan)
    if not np.isnan(X):
        X_values.append(X)
        L_inv_values.append(1.0/L)
        print(f"L = {L:3d}, 1/L = {1.0/L:.4f}, X = {X:.3f} ± {summaries_delta1[L].get('fit_X_mi_err', 0):.3f}")

# --- Plotting ---
print("\nGenerating plots...")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), constrained_layout=True)
colors = plt.cm.viridis(np.linspace(0, 0.8, len(L_values)))
markers = ['o', '^', 'P', 'X']

# --- Left: I(r) log–log ---
for i, L in enumerate(L_values):
    if L in results_delta1 and results_delta1[L]:
        r1, I1 = results_delta1[L]['r_values_mi'], results_delta1[L]['I_r_values']
        ax1.loglog(r1, I1, marker=markers[i % len(markers)], ls='-', color=colors[i], label=f'Δ=1.0 (L={L})')
    if L in results_delta2 and results_delta2[L] and L == max(L_values):
        r2, I2 = results_delta2[L]['r_values_mi'], results_delta2[L]['I_r_values']
        ax1.loglog(r2, I2, 's--', color='orange', label=f'Δ=2.0 (L={L})')

if max(L_values) in results_delta1 and results_delta1[max(L_values)]:
    r_ref = np.array(results_delta1[max(L_values)]['r_values_mi'], dtype=float)
    ref_idx = min(fit_r_min - 1, len(results_delta1[max(L_values)]['I_r_values']) - 1)
    I_ref_start = results_delta1[max(L_values)]['I_r_values'][ref_idx] 
    r_ref_start = r_ref[ref_idx]
    C_ref = I_ref_start * (r_ref_start**2)
    ax1.loglog(r_ref, C_ref * r_ref**(-2), 'k:', label=r'$I(r) \propto r^{-2}$ ref', alpha=0.7)

ax1.set_title("Mutual Information $I(r)$")
ax1.set_xlabel("Distance $r$")
ax1.set_ylabel("$I(r)$")
ax1.grid(True, which="both", ls="--", alpha=0.5)
ax1.legend()

# --- Right: d_E(r) linear ---
for i, L in enumerate(L_values):
    if L in results_delta1 and results_delta1[L]:
        r1, dE1 = results_delta1[L]['r_values_mi'], results_delta1[L]['d_E_r_values']
        ax2.plot(r1, dE1, marker=markers[i % len(markers)], ls='-', color=colors[i], label=f'Δ=1.0 (L={L})')
    if L in results_delta2 and results_delta2[L] and L == max(L_values):
        r2, dE2 = results_delta2[L]['r_values_mi'], results_delta2[L]['d_E_r_values']
        ax2.plot(r2, dE2, 's--', color='orange', label=f'Δ=2.0 (L={L})')

if max(L_values) in results_delta1 and results_delta1[max(L_values)]:
    r_ref = np.array(results_delta1[max(L_values)]['r_values_mi'], dtype=float)
    k_ref = summaries_delta1.get(max(L_values), {}).get('fit_k_de', 1.0)
    ax2.plot(r_ref, k_ref * r_ref, 'k:', label=f'$d_E = {k_ref:.2f} \\times r$ ref', alpha=0.7)

ax2.set_title("Emergent Distance $d_E(r)$") 
ax2.set_xlabel("Distance $r$")
ax2.set_ylabel("$d_E(r)$")
ax2.grid(True, ls="--", alpha=0.5)
ax2.legend()

plot_file = os.path.join(outdir, "prl_jewel_plot_finite_size.png")
plt.savefig(plot_file, dpi=300)
print(f"Saved plot to {plot_file}")

plt.show()

print("\n--- Recommendations for Next Steps ---")
print("1. Analyze Convergence & Tolerance.")
print("2. Check File Outputs: Make sure your runs aren't overwriting each other's data!")
print("3. Finite-Size Extrapolation.")
print("4. Truncation Error Checks.")
print("5. Supplemental Material.")

# ==============================================================================
# NEW CODE BLOCK: SAVE THE COMPLETE, RAW DATA TO CSV
# Add this to the end of your existing, working Jupyter cell.
# ==============================================================================

# Define the full, absolute path for the output directory
output_directory = "/Users/beautrudel/Desktop/Projects/Project-Photon/Jewel_Test/PRL_Golden_Run"
# Ensure the directory exists
os.makedirs(output_directory, exist_ok=True)

print(f"\n--- Saving complete data sets to: {output_directory} ---")

# --- Save data for Delta = 1.0 runs ---
for L, data in results_delta1.items():
    if data: # Check if data dictionary is not empty
        output_filename = os.path.join(output_directory, f"paper1_final_data_L{L}_delta1.0.csv")
        # Extract the arrays we need for the plot
        df = pd.DataFrame({
            'r': data['r_values_mi'],
            'I(r)': data['I_r_values'],
            'd_E(r)': data['d_E_r_values']
        })
        df.to_csv(output_filename, index=False, float_format='%.10e')
        print(f"--> Successfully saved: {output_filename}")

# --- Save data for Delta = 2.0 runs ---
for L, data in results_delta2.items():
    if data: # Check if data dictionary is not empty
        output_filename = os.path.join(output_directory, f"paper1_final_data_L{L}_delta2.0.csv")
        df = pd.DataFrame({
            'r': data['r_values_mi'],
            'I(r)': data['I_r_values'],
            'd_E(r)': data['d_E_r_values']
        })
        df.to_csv(output_filename, index=False, float_format='%.10e')
        print(f"--> Successfully saved: {output_filename}")

print("\n--- Raw data export complete. ---")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def generate_figure_with_inset_v2():
    """
    Creates the main 2-panel figure with the inset shifted to the right.
    """
    # --- Load Data ---
    data_path = "/Users/beautrudel/Desktop/Projects/Project-Photon/Jewel_Test/PRL_Golden_Run"

    try:
        df_l64_d1 = pd.read_csv(os.path.join(data_path, 'paper1_final_data_L64_delta1.0.csv'))
        df_l64_d2 = pd.read_csv(os.path.join(data_path, 'paper1_final_data_L64_delta2.0.csv'))
        df_l96_d1 = pd.read_csv(os.path.join(data_path, 'paper1_final_data_L96_delta1.0.csv'))
        df_l96_d2 = pd.read_csv(os.path.join(data_path, 'paper1_final_data_L96_delta2.0.csv'))
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure the path in the 'data_path' variable is correct.")
        return

    # --- Calculate Emergent Distance and K_0 ---
    for df in [df_l96_d1, df_l64_d1, df_l96_d2, df_l64_d2]:
        df['d_E_raw'] = 1.0 / np.sqrt(df['I(r)'])

    calib_data = df_l96_d1[(df_l96_d1['r'] >= 10) & (df_l96_d1['r'] <= 45)]
    slope_calib, _ = np.polyfit(calib_data['r'], calib_data['d_E_raw'], 1)
    K0 = 1.0 / slope_calib

    # --- Generate Main Plot ---
    plt.style.use('default')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5.5))
    plt.rcParams['font.family'] = 'serif'
    colors = {'l96_d1': '#2ca02c', 'l64_d1': '#9467bd', 'l96_d2': '#ff7f0e', 'l64_d2': '#d62728'}

    # Panel (a): Mutual Information
    ax1.loglog(df_l96_d1['r'], df_l96_d1['I(r)'], '^-', markersize=5, color=colors['l96_d1'], label='$\Delta=1.0$ (L=96)')
    ax1.loglog(df_l64_d1['r'], df_l64_d1['I(r)'], 'o-', markersize=5, color=colors['l64_d1'], label='$\Delta=1.0$ (L=64)')
    ax1.loglog(df_l96_d2['r'], df_l96_d2['I(r)'], 's--', markersize=5, color=colors['l96_d2'], label='$\Delta=2.0$ (L=96)')
    ax1.loglog(df_l64_d2['r'], df_l64_d2['I(r)'], 'x--', markersize=5, color=colors['l64_d2'], label='$\Delta=2.0$ (L=64)')
    r_ref = np.logspace(np.log10(5), np.log10(50), 100)
    ax1.plot(r_ref, (K0**-2) * r_ref**-2, 'k:', label='$I(r) \propto r^{-2}$ ref')
    ax1.set(xlabel='Distance r', ylabel='Mutual Information $I(r)$', title='(a) Mutual Information')
    ax1.legend()
    ax1.grid(True, which="both", ls="--", alpha=0.5)

    # Panel (b): Emergent Distance
    ax2.plot(df_l96_d1['r'], K0 * df_l96_d1['d_E_raw'], '^-', markersize=5, color=colors['l96_d1'])
    ax2.plot(df_l64_d1['r'], K0 * df_l64_d1['d_E_raw'], 'o-', markersize=5, color=colors['l64_d1'])
    ax2.plot(df_l96_d2['r'], K0 * df_l96_d2['d_E_raw'], 's--', markersize=5, color=colors['l96_d2'])
    ax2.plot(df_l64_d2['r'], K0 * df_l64_d2['d_E_raw'], 'x--', markersize=5, color=colors['l64_d2'])
    r_ref_linear = np.array([0, 50])
    ax2.plot(r_ref_linear, r_ref_linear, 'k:', label='$d_E(r) = r$ ref')
    ax2.set(xlabel='Distance r', ylabel='Emergent Distance $d_E(r)$', title='(b) Emergent Distance')
    max_y = max((K0 * df_l96_d2['d_E_raw']).max(), (K0 * df_l64_d2['d_E_raw']).max())
    ax2.set_xlim(0, 50)
    ax2.set_ylim(0, max_y * 1.05)
    ax2.legend()
    ax2.grid(True, which="both", ls="--", alpha=0.5)

    # --- THIS IS THE MODIFIED INSET ---
    # The first coordinate is changed from 0.45 to 0.52 to shift it right.
    ax_inset = ax2.inset_axes([0.52, 0.1, 0.45, 0.4]) # [left, bottom, width, height]
    ax_inset.plot(df_l64_d1['r'], K0 * df_l64_d1['d_E_raw'], 'o-', markersize=4, color=colors['l64_d1'])
    ax_inset.plot(df_l64_d2['r'], K0 * df_l64_d2['d_E_raw'], 'x--', markersize=4, color=colors['l64_d2'])
    ax_inset.plot(r_ref_linear, r_ref_linear, 'k:')
    ax_inset.set_title('Zoom: $L=64$', fontsize=10)
    ax_inset.set_xlim(0, 40)
    max_y_inset = (K0 * df_l64_d2[df_l64_d2['r']<=40]['d_E_raw']).max()
    ax_inset.set_ylim(0, max_y_inset * 1.1)
    ax_inset.grid(True, which="both", ls="--", alpha=0.3)

    # --- Save Figure ---
    fig.tight_layout()
    output_filename = 'figure_1_with_inset_v2.png'
    plt.savefig(output_filename, dpi=300)
    print(f"\nPlot saved successfully as '{output_filename}'")

if __name__ == '__main__':
    generate_figure_with_inset_v2()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.optimize import curve_fit

def generate_exponent_drift_plot():
    """
    Creates a plot showing the fitted exponent X as a function of the
    starting point of the fit window, r_min.
    """
    # --- Load Data ---
    data_path = "/Users/beautrudel/Desktop/Projects/Project-Photon/Jewel_Test/PRL_Golden_Run"

    try:
        df = pd.read_csv(os.path.join(data_path, 'paper1_final_data_L96_delta1.0.csv'))
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure the path in the 'data_path' variable is correct.")
        return

    # --- Define Fit Function ---
    def power_law(r, C, X):
        return C * r**(-X)

    # --- Iterate through fit windows ---
    r_min_values = range(2, 31) # Fit windows will start from r=2 up to r=30
    exponents = []
    errors = []

    for r_min in r_min_values:
        r_max = 45 # Keep the end of the fit window fixed
        fit_data = df[(df['r'] >= r_min) & (df['r'] <= r_max)]

        if len(fit_data) < 3: # Need at least 3 points to fit
            continue

        try:
            params, cov = curve_fit(power_law, fit_data['r'], fit_data['I(r)'], p0=[1.0, 2.0])
            X_val = params[1]
            X_err = np.sqrt(np.diag(cov))[1]
            exponents.append(X_val)
            errors.append(X_err)
        except RuntimeError:
            # If fit fails, append NaN or skip
            exponents.append(np.nan)
            errors.append(np.nan)

    # Filter out failed fits for plotting
    valid_indices = ~np.isnan(exponents)
    plot_rmin = np.array(r_min_values)[valid_indices]
    plot_exponents = np.array(exponents)[valid_indices]
    plot_errors = np.array(errors)[valid_indices]


    # --- Generate Plot ---
    plt.style.use('default')
    fig, ax = plt.subplots(figsize=(8, 5))
    plt.rcParams['font.family'] = 'serif'

    ax.errorbar(plot_rmin, plot_exponents, yerr=plot_errors, fmt='o-', capsize=4, label='Fitted Exponent $X$')
    
    # Add horizontal line for the ideal X=2 case
    ax.axhline(2.0, color='k', linestyle='--', label='$X=2$ (Ideal Euclidean Limit)')
    
    # Add horizontal lines for the observed range
    ax.axhline(1.5, color='r', linestyle=':', alpha=0.7, label='$X \\approx 1.5$ (Long-distance fit)')


    # --- Final Touches ---
    ax.set_xlabel('Start of Fitting Window, $r_\mathrm{min}$')
    ax.set_ylabel('Fitted Information Exponent, $X$')
    ax.set_title('Exponent Drift vs. Fitting Window ($L=96, \Delta=1.0$)')
    ax.legend()
    ax.grid(True, which="both", ls="--", alpha=0.5)
    ax.set_xticks(range(2, 31, 2)) # Set x-axis ticks to be integers
    
    # --- Save Figure ---
    fig.tight_layout()
    output_filename = 'exponent_vs_fit_window.png'
    plt.savefig(output_filename, dpi=300)
    print(f"\nPlot saved successfully as '{output_filename}'")


if __name__ == '__main__':
    generate_exponent_drift_plot()